## Applying Romer-Romer Monetary Policy Shock to S&P500 Notebook

Set System Path and Import Modules

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/bradleychao/Desktop/EC15_Research'))
if module_path not in sys.path:
    sys.path.append('/Users/bradleychao/Desktop/EC15_Research')

import numpy as np
import pandas as pd

Extract historical S&P 500 data from Bloomberg terminal and combine with Wielan's updated Romer-Romer MPS 

In [2]:
# Obtain full dataset which runs from January 1969 to May 2023
df_fullspx = pd.read_excel('Daily_SPX_since_1969.xlsx')

# Convert 'Date' column to datetime format
df_fullspx['Date'] = pd.to_datetime(df_fullspx['Date'])

# Cut dataset off at December 2007
end_date = pd.to_datetime('2007-12-31')
df_filtered = df_fullspx[df_fullspx['Date'] <= end_date]

# Compute monthly averages
df_filtered.set_index(['Date'], inplace=True)
monthly_SPX = df_filtered[['PX_LAST']].resample('M').mean()

# Get rid of temporary indexing
monthly_SPX.reset_index(drop=True, inplace=True)

monthly_SPX

,PX_LAST
0,102.043182
1,101.468889
2,99.304500
3,101.262857
4,104.621905
...,...
463,1454.616957
464,1497.117368
465,1539.659565
466,1463.385238


In [3]:
# Obtain Wielan's dataset
df_wielan = pd.read_stata('MS1/RR_monetary_shock_monthly.dta')
df_wielan = df_wielan.loc[:, ['date', 'resid_full']]
df_wielan.rename(columns = {'date': 'Date'}, inplace = True)

df_wielan

,Date,resid_full
0,1969-01-01,0.000000
1,1969-02-01,0.000000
2,1969-03-01,-0.227409
3,1969-04-01,0.464021
4,1969-05-01,0.209193
...,...,...
463,2007-08-01,0.024113
464,2007-09-01,-0.587556
465,2007-10-01,-0.317340
466,2007-11-01,0.000000


In [4]:
combined_data = pd.DataFrame()

combined_data['Romer'] = df_wielan['resid_full']
combined_data['SPX'] = monthly_SPX['PX_LAST']

# Create log_SP500
combined_data['log_SP500'] = np.log(combined_data['SPX'])
combined_data.drop('SPX', axis=1, inplace=True)

combined_data

,Romer,log_SP500
0,0.000000,4.625396
1,0.000000,4.619752
2,-0.227409,4.598191
3,0.464021,4.617720
4,0.209193,4.650353
...,...,...
463,0.024113,7.282498
464,-0.587556,7.311297
465,-0.317340,7.339317
466,0.000000,7.288508


In [5]:
# Full dataset ranges from 1969 to 2007
full_dataset = combined_data

# Original Romer-Romer ranges from 1969 to 1996
original_dataset = combined_data.iloc[0:336]

full_dataset.to_stata("full_dataset.dta")
original_dataset.to_stata("original_dataset.dta")

      Romer  log_SP500
0  0.000000   4.625396
1  0.000000   4.619752
2 -0.227409   4.598191
3  0.464021   4.617720
4  0.209193   4.650353
